In [ ]:
import pandas as pd
import scipy
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import chardet
with open('algal.csv', 'rb') as f:
    enc = chardet.detect(f.read())  # or readline if the file is large

df=pd.read_csv('algal.csv', encoding = enc['encoding'])

FileNotFoundError: [Errno 2] No such file or directory: 'algal.csv'

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:

df.describe()

In [ ]:
df.dtypes

 #df.fillna(df.mean(), inplace=True)

In [ ]:
print(df)

In [ ]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Handle null values for object columns
object_cols = df.select_dtypes(include=['object']).columns
df[object_cols] = df[object_cols].fillna(df[object_cols].mode().iloc[0])

In [ ]:
df = df.drop(columns=['Time of Occurrence'])

In [ ]:


def remove_outliers(df, threshold=3):
    """
    Remove outliers from a DataFrame

    Parameters:
        df (DataFrame): Input DataFrame
        threshold (float): Z-score threshold for identifying outliers in numeric columns

    Returns:
        DataFrame: DataFrame with outliers removed
    """
    # Copy the original DataFrame to avoid modifying the original data
    cleaned_df = df.copy()

    # Handle numeric columns
    numeric_cols = cleaned_df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_cols:
        z_scores = np.abs((cleaned_df[col] - cleaned_df[col].mean()) / cleaned_df[col].std())
        outliers = cleaned_df[z_scores > threshold]
        cleaned_df = cleaned_df.drop(outliers.index)

    # Handle object columns
    object_cols = cleaned_df.select_dtypes(include=['object']).columns
    for col in object_cols:
        # Define your own criteria for identifying outliers for object columns
        # For example, remove rows with 'NaN' or specific strings
        outliers = cleaned_df[cleaned_df[col].isin(['NaN', 'outlier_value'])]
        cleaned_df = cleaned_df.drop(outliers.index)

    return cleaned_df

# Example usage:
cleaned_df = remove_outliers(df)


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
numeric_cols = cleaned_df.select_dtypes(include=['int64', 'float64']).columns
object_cols = cleaned_df.select_dtypes(include=['object']).columns

# Define preprocessing steps for numeric and object columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values if any
    ('scaler', StandardScaler())  # Standardize numeric features
])

object_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values if any
    ('onehot', OneHotEncoder())  # One-hot encode categorical features
])

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', object_transformer, object_cols)])

# Fit and transform the data
processed_data = preprocessor.fit_transform(cleaned_df)

# Get transformed numeric and object column names
numeric_transformed_cols = preprocessor.transformers_[0][1]['scaler'].get_feature_names_out(numeric_cols)
object_transformed_cols = preprocessor.transformers_[1][1]['onehot'].get_feature_names_out(object_cols)
print(processed_data.shape)
print(len(numeric_transformed_cols))
print(len(object_transformed_cols))
# Combine transformed numeric and object columns into a DataFrame
# Combine transformed numeric and object columns into a DataFrame
#processed_df = pd.DataFrame(processed_data, columns=list(numeric_transformed_cols) + list(object_transformed_cols.flatten()))
# Combine transformed numeric and object columns into a DataFrame
numeric_and_object_transformed_cols = list(numeric_transformed_cols) + list(object_transformed_cols.flatten())
#processed_df = pd.DataFrame(processed_data, columns=numeric_and_object_transformed_cols)
print("Numeric Transformed Columns Shape:", numeric_transformed_cols.shape)
print("Object Transformed Columns Shape:", object_transformed_cols.shape)
print("Shape of processed_data:", processed_data.shape)
print("Length of concatenated column names:", len(numeric_and_object_transformed_cols))

print("Column Names:", processed_df.columns)
# Create DataFrame from processed_data using ColumnTransformer
processed_df = pd.DataFrame(processed_data, columns=numeric_and_object_transformed_cols)


In [ ]:
column_names = cleaned_df.columns.tolist()

print("Column Names:", column_names)

In [ ]:
print("Processed Data Shape:", processed_data.shape)
print("Processed Data Sample:")
print(processed_data[:5])  # Print the first 5 rows of the processed data


In [ ]:
print("Shape of processed_data:", processed_data.shape)
print("Length of combined column names:", len(numeric_and_object_transformed_cols))


In [ ]:
# Create DataFrame from processed_data
processed_df = pd.DataFrame(processed_data)

# Assign column names to the DataFrame
processed_df.columns = numeric_and_object_transformed_cols


In [ ]:
# Create DataFrame from processed_data with no column names initially
processed_df = pd.DataFrame(processed_data, columns=None)

# Assign column names to the DataFrame
processed_df.columns = numeric_and_object_transformed_cols


In [66]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load your DataFrame
#cleaned_df = pd.read_csv("your_data.csv")

# Separate numerical and object columns
numerical_cols = cleaned_df.select_dtypes(include=['float64', 'int64']).columns
object_cols = cleaned_df.select_dtypes(include=['object']).columns

# Convert object columns to numerical format
for col in object_cols:
    label_encoder = LabelEncoder()
    cleaned_df[col] = label_encoder.fit_transform(cleaned_df[col])

# Standardize numerical columns
scaler_numerical = StandardScaler()
cleaned_df[numerical_cols] = scaler_numerical.fit_transform(cleaned_df[numerical_cols])

# Display standardized DataFrame
print(cleaned_df.head())




   Latitude  Longitude       SST  Chlorophyll a       par  Cells per litre  \
0  0.758882   2.537251 -2.968318      -0.345506 -1.435455        -0.374098   
2  0.758882   0.886974 -0.015082      -1.833051  0.654619         0.105654   
3  0.127591  -0.427070  0.536083      -0.007427 -0.619562        -0.457255   
6  1.120805   0.586238 -0.054590       0.398267 -0.955994         1.544909   
7 -3.304400  -0.383066  0.653476       0.195420  0.810569        -0.342115   

   Day of year  Temperature  Dew point Temperature  Sea level pressure(hPa)  \
0     0.145822     0.132102               0.614062                -0.519737   
2    -0.061151     0.662246              -0.472725                -0.519192   
3     1.010238    -0.842217              -0.627980                -0.518022   
6    -0.097676     0.733887              -2.232284                -0.518730   
7    -2.009131    -0.355057              -0.330407                -0.520009   

   ...     a_443     a_488     a_547     a_645     a_667